In [1]:
# Correlation matrices saved as Excel file
# interaction map 8d, bins, shape modes
# concordance 8d, bins, shape modes
# concordance edge, non-edge, i1, m1, i2, m2 

In [2]:
!pwd
!date

/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_notebooks/Tables
Mon May 30 19:27:49 PDT 2022


In [3]:
import os
import sys
import importlib
import concurrent
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from skimage import io as skio
import matplotlib.pyplot as plt
from aicscytoparam import cytoparam
from aicsshparam import shtools, shparam
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from cvapipe_analysis.tools import io, viz, general, controller, shapespace, plotting

sys.path.insert(1, '../tools')
import common

In [4]:
# Controller form cvapipe_analysis
path_config = Path("/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/")
config = general.load_config_file(path_config)
control = controller.Controller(config)
device = io.LocalStagingIO(control)
df = device.load_step_manifest("loaddata")
print(df.shape, control.get_staging())

/allen/aics/assay-dev/MicroscopyOtherData/Viana/projects/cvapipe_analysis/local_staging_variance


In [5]:
# Removing bad NUP plate. This has to be fixed when paper is submitted <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
df = df.loc[df.PlateId!=3500002920]
df_size = df.groupby("structure_name").size()
print(df.shape)

In [15]:
# Number of FOVs
df.FOVId.nunique()

  0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
# Number of outliers outliers
df_out = pd.read_csv("../../local_staging_variance/preprocessing/outliers.csv", index_col="CellId")
df_out.loc[df_out.Outlier!="No"].shape

(1044, 2)

In [9]:
# Mitotic vs interphase
df_int = df.loc[df.cell_stage=="M0"]
df_int_size = df_int.groupby("structure_name").size()
df_mito = df.loc[df.cell_stage!="M0"]
df_mito_size = df_mito.groupby("structure_name").size()

In [10]:
# Analysis dataset (interphase + no outliers)
df_analysis = device.load_step_manifest("preprocessing")
df_analysis_size = df_analysis.groupby("structure_name").size()
print(df_analysis.shape)

(202847, 1290)


In [11]:
# Shape modes dataset (No cells between 0-1th and 99-100th percentiles) 
space = shapespace.ShapeSpace(control)
space.execute(df_analysis)
df_shapemodes = space.shape_modes
print(df_shapemodes.shape)

(175148, 8)


In [12]:
# Cells inside the 8d sphere
df_sphere_size = []
df_sphere = space.get_cells_inside_ndsphere_of_radius()

In [13]:
df_sphere_size = pd.Series(dict([(row.structure, len(row.CellIds)) for index, row in df_sphere.iterrows()]), name="structure_name")

### Other datasets

In [14]:
importlib.reload(common)
dsname = "edges"
path_cvapipe = Path(control.get_staging()).parent
datasets = {
    "edges": {
        "control": f"{path_cvapipe}/local_staging_variance_edges",
        "perturbed": f"{path_cvapipe}/local_staging_edge_cells_midpoint_expanded"
    },
    "m1m2": {
        "control": f"{path_cvapipe}/local_staging_variance_m1m2",
        "perturbed": f"{path_cvapipe}/local_staging_m1m2"
    },
    "m3": {
        "control": f"{path_cvapipe}/local_staging_variance_m3",
        "perturbed": f"{path_cvapipe}/local_staging_m3"
    }}
managers = {}
for dsname, ds in datasets.items():
    managers[dsname] = common.setup_cvapipe_for_matched_dataset(config, ds)

In [15]:
# Edge cells dataset
df_edge_ct = managers["edges"]["control"]["device"].load_step_manifest("preprocessing")
df_edge_pt = managers["edges"]["perturbed"]["device"].load_step_manifest("preprocessing")
df_edge_ct_size = df_edge_ct.groupby("structure_name").size()
df_edge_pt_size = df_edge_pt.groupby("structure_name").size()
print(df_edge_ct.shape, df_edge_pt.shape)

(4477, 1290) (5169, 1255)


In [16]:
# M1 cells dataset
df_m1m2_ct = managers["m1m2"]["control"]["device"].load_step_manifest("preprocessing")
df_m1m2_pt = managers["m1m2"]["perturbed"]["device"].load_step_manifest("preprocessing")
df_m1m2_ct_size = df_m1m2_ct.groupby("structure_name").size()
df_m1m2_pt_size = df_m1m2_pt.groupby("structure_name").size()
print(df_m1m2_ct.shape, df_m1m2_pt.shape)

(1969, 1290) (2201, 1255)


In [18]:
# M2 cells dataset
df_m3_ct = managers["m3"]["control"]["device"].load_step_manifest("preprocessing")
df_m3_pt = managers["m3"]["perturbed"]["device"].load_step_manifest("preprocessing")
df_m3_ct_size = df_m3_ct.groupby("structure_name").size()
df_m3_pt_size = df_m3_pt.groupby("structure_name").size()
print(df_m3_ct.shape, df_m3_pt.shape)

(877, 1290) (981, 1255)


### Merging all columns together

In [32]:
df_ds = pd.DataFrame([
    df_size,
    df_int_size,
    df_mito_size,
    df_analysis_size,
    df_sphere_size,
    df_edge_ct_size,
    df_edge_pt_size,
    df_m1m2_ct_size,
    df_m1m2_pt_size,
    df_m3_ct_size,
    df_m3_pt_size
]).T
df_ds.columns=pd.MultiIndex.from_tuples([
    ("single cell dataset", "total"),
    ("single cell dataset", "interphase"),
    ("single cell dataset", "mitotic"),
    ("single cell dataset", "analysis"),
    ("single cell dataset", "8d sphere"),
    ("shape-matched dataset","non-edge cells"),
    ("shape-matched dataset","edge cells"),
    ("shape-matched dataset","interphase"),
    ("shape-matched dataset","prophase"),
    ("shape-matched dataset","interphase"),
    ("shape-matched dataset","early prometaphase")
])
df_ds = df_ds.loc[control.get_gene_names()]

In [33]:
df_work = df.groupby("structure_name").first()
df_work = df_work.WorkflowId
for k, v in df_work.items():
    df_work[k] = v.split("'")[1]

In [34]:
df_ds.insert(loc=0, column="Workflow", value=df_work)
# Order that we want to display (by acquisition)
df_ds.insert(loc=1, column="Acquisition Order", value=[10,17,25,24,21,3,22,5,23,20,19,2,11,7,1,12,13,9,4,18,14,6,15,8,16])
df_ds.insert(loc=2, column="Structure", value=control.get_structure_names())

In [35]:
df_ds = df_ds.append(df_ds.sum().rename('Total'))
df_ds

Workflow  \
                                                               
FBL                                             Pipeline 4.1   
NPM1                                            Pipeline 4.2   
SON                                             Pipeline 4.4   
SMC1A                                           Pipeline 4.4   
HIST1H2BJ                                       Pipeline 4.4   
LMNB1                                             Pipeline 4   
NUP153                                          Pipeline 4.4   
SEC61B                                            Pipeline 4   
ATP2A2                                          Pipeline 4.4   
SLC25A17                                        Pipeline 4.4   
RAB5A                                           Pipeline 4.4   
TOMM20                                            Pipeline 4   
LAMP1                                           Pipeline 4.1   
ST6GAL1                                         Pipeline 4.1   
TUBA1B                                            Pipeline 4   
CETN2                                           Pipeline 4.2   
GJA1                                            Pipeline 4.2   
TJP1                                            Pipeline 4.1   
DSP                                               Pipeline 4   
CTNNB1                                          Pipeline 4.2   
AAVS1                                           Pipeline 4.2   
ACTB                                            Pipeline 4.1   
ACTN1                                           Pipeline 4.2   
MYH10                                           Pipeline 4.1   
PXN                                             Pipeline 4.2   
Total      Pipeline 4.1Pipeline 4.2Pipeline 4.4Pipeline 4...   

          Acquisition Order  \
                              
FBL                      10   
NPM1                     17   
SON                      25   
SMC1A                    24   
HIST1H2BJ                21   
LMNB1                     3   
NUP153                   22   
SEC61B                    5   
ATP2A2                   23   
SLC25A17                 20   
RAB5A                    19   
TOMM20                    2   
LAMP1                    11   
ST6GAL1                   7   
TUBA1B                    1   
CETN2                    12   
GJA1                     13   
TJP1                      9   
DSP                       4   
CTNNB1                   18   
AAVS1                    14   
ACTB                      6   
ACTN1                    15   
MYH10                     8   
PXN                      16   
Total                   325   

                                                   Structure  \
                                                               
FBL                                           nucleoli [DFC)   
NPM1                                           nucleoli [GC)   
SON                                         nuclear speckles   
SMC1A                                               cohesins   
HIST1H2BJ                                           histones   
LMNB1                                       nuclear envelope   
NUP153                                         nuclear pores   
SEC61B                                       ER [Sec61 beta)   
ATP2A2                                           ER [SERCA2)   
SLC25A17                                         peroxisomes   
RAB5A                                              endosomes   
TOMM20                                          mitochondria   
LAMP1                                              lysosomes   
ST6GAL1                                                Golgi   
TUBA1B                                          microtubules   
CETN2                                             centrioles   
GJA1                                           gap junctions   
TJP1                                         tight junctions   
DSP                                               desmosomes   
CTNNB1                                    adhe

In [36]:
df_ds.to_csv("TableS1_main.csv")

### Number of cells along shape modes

In [171]:
df_bin = []
mpIdc = control.get_center_map_point_index()
for sm in control.get_shape_modes():
    space.set_active_shape_mode(sm, digitize=True)
    for mpId in control.get_map_point_indexes():
        space.set_active_map_point_index(mpId)
        serie_bin = df.loc[space.get_active_cellids()].groupby("structure_name").size()
        serie_bin.name = (sm, mpId-mpIdc)
        df_bin.append(serie_bin)
df_bin = pd.DataFrame(df_bin).T
df_bin = df_bin.loc[control.get_gene_names()]

In [173]:
df_bin["Structure"] = control.get_structure_names()
df_bin.set_index("Structure", drop=True)

NUC_MEM_PC1                                                 \
                            -4    -3    -2    -1     0     1     2     3    4   
Structure                                                                       
nucleoli [DFC)              94   295   642  1096  1530  1761  1488  1015  502   
nucleoli [GC)              123   385   887  1443  1854  2078  1695  1111  424   
nuclear speckles           580   524   383   293   252   157    74    39    7   
cohesins                   193   336   430   467   334   172    85    28    6   
histones                   748  1490  2314  2554  2289  2045  1355   586  133   
nuclear envelope           829  1374  1802  2025  1900  1457   833   348  109   
nuclear pores              382   952  1918  2633  3111  2786  1914   913  309   
ER [Sec61 beta)             97   173   356   593   814   942   934   798  511   
ER [SERCA2)                683  1478  2140  2078  1430   675   279   116   17   
peroxisomes                 29   122   225   317   338   305   176    74   29   
endosomes                  148   352   420   365   375   271   155    61   20   
mitochondria               514  1260  2226  3234  3860  3756  2932  1584  759   
lysosomes                  370   773  1277  1483  1512  1327  1000   706  317   
Golgi                      181   392   750   966   901   780   658   496  216   
microtubules                88   233   537   988  1272  1446  1427  1128  595   
centrioles                 225   502   707   873   928   852   811   681  334   
gap junctions              142   322   591   842   932   848   727   498  324   
tight junctions             94   206   457   600   837   861   756   564  273   
desmosomes                 160   345   706  1089  1423  1467  1377   936  479   
adherens junctions         333   572   800   914   886   778   552   212   63   
plasma membrane             84   246   604  1158  1440  1270   972   622  278   
actin filaments              8    54   145   248   356   518   627   650  498   
actin bundles              162   447   753  1058  1177  1132   863   605  286   
actomyosin bundles         209   470   686   996  1030   849   538   331  175   
matrix adhesions           400   554   592   560   387   241   154    80   23   

                   NUC_MEM_PC2  ... NUC_MEM_PC7 NUC_MEM_PC8                    \
                            -4  ...           4          -4    -3    -2    -1   
Structure                       ...                                             
nucleoli [DFC)             240  ...         416         409   480   870  1495   
nucleoli [GC)              189  ...         467         509   624  1138  1804   
nuclear speckles            40  ...          95          58    96   246   461   
cohesins                    48  ...          78          62    86   213   351   
histones                   509  ...         600         622   798  1639  2491   
nuclear envelope           157  ...         396         304   477  1122  2002   
nuclear pores              597  ...         615         753   903  1745  2791   
ER [Sec61 beta)            129  ...         238         323   354   594   898   
ER [SERCA2)                405  ...         348         243   379   842  1532   
peroxisomes                 88  ...          68          75   110   186   290   
endosomes                   44  ...          97          95   119   277   418   
mitochondria               356  ...         920         843  1076  2170  3427   
lysosomes                  170  ...         389         347   453   936  1589   
Golgi                       77  ...         226         227   281   576   983   
microtubules               226  ...         350         351   421   856  1434   
centrioles                 131  ...         232         261   291   638  1114   
gap junctions              103  ...         237         249   284   512   904   
tight junctions            125  ...         236         253   264   496   792   
desmosomes                 181  ...         422  

In [174]:
df_bin.to_csv("TableS1_shapemodes.csv")